In [3]:
import os
import sys
sys.path.insert(1, 'C:/Users/BGH360/Desktop/KilianLiss/UniSydney/Research/20210208MovingPlates/pyGplates/pygplates_rev28_python38_win64')

import pygplates
import numpy as np
import pyvista as pv

## Coordinate Transformations

Begin the project by defining a bunch of functions related to coordinate transformations. Most of these are related to transformations between the spherical polar coordinate system, and a cartesian (XYZ) coordinate system.

I also include a function for creating quaternions used in rotation (moving tectonic plates), and for normalizing arrays (transforming their values into a desired range).

In [2]:
#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(R, Theta, Phi):
    X = R * np.cos(Theta) * np.sin(Phi)
    Y = R * np.sin(Theta) * np.sin(Phi)
    Z = R * np.cos(Phi)
    return X, Y, Z

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(X, Y, Z):
    R = (X**2 + Y**2 + Z**2)**0.5
    Theta = np.arctan2(Y, X)
    Phi = np.arccos(Z / R)
    return R, Theta, Phi

#Takes longatude and latitude coordinates and converts them to cartesian coordinates
def lonLatToCartesian(lon, lat, radius=1.0):
    X, Y, Z = polarToCartesian(radius, np.radians(lon+180), np.radians(90 - lat))
    return np.stack((X, Y, Z), axis=-1)

#Takes cartesian coordinates and converts them to longatude and latitude
def cartesianToLonLat(X, Y, Z):
    r, theta, phi = cartesianToPolarCoords(X, Y, Z)
    theta, phi = np.degrees(theta), np.degrees(phi)
    lon, lat = theta - 180, 90 - phi
    lon[lon < -180] = lon[lon < -180] + 360
    return lon, lat

#Function for moving vertices on a sphere along the radial direction by amount of delta radius (dr)
def moveAlongRadialDirection(XYZ, dr):
    r, theta, phi = cartesianToPolarCoords(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2])
    newSphereX = XYZ[:, 0] + np.cos(theta) * np.sin(phi) * dr
    newSphereY = XYZ[:, 1] + np.sin(theta) * np.sin(phi) * dr
    newSphereZ = XYZ[:, 2] + np.cos(phi) * dr
    return np.stack((newSphereX, newSphereY, newSphereZ), axis=-1)

#Given a sphere XYZ coordinates, we set the radius of all coordinates
def setRadialComponent(XYZ, r):
    rad, theta, phi = cartesianToPolarCoords(XYZ[:, 0], XYZ[:, 1], XYZ[:, 2])
    newSphereX = np.cos(theta) * np.sin(phi) * r
    newSphereY = np.sin(theta) * np.sin(phi) * r
    newSphereZ = np.cos(phi) * r
    return np.stack((newSphereX, newSphereY, newSphereZ), axis=-1)

#Returns a rotation quaternion
def quaternion(axis, angle):
    return [np.sin(angle/2) * axis[0], 
            np.sin(angle/2) * axis[1], 
            np.sin(angle/2) * axis[2], 
            np.cos(angle/2)]

#Normalizes the height map or optionally brings the heightmap within specified height limits
def normalizeArray(A, minValue=0.0, maxValue=1.0):
    A = A - min(A)
    A = A / (max(A) - min(A))
    return A * (maxValue - minValue) + minValue